In [1]:

%load_ext autoreload
%autoreload 2

import os
import cv2
import numpy as np
import torch
from tqdm import tqdm
from imageio import get_writer
from collections import OrderedDict

In [14]:
import util
from models  import load_model
from data    import load_data
from options import base_options
#from pose    import load_estimator

In [70]:
opt            = base_options.Options().parse(save=False)
opt.isTrain    = False
opt.name       = 'everybody_dance_now_temporal'
opt.model      = 'pose2vid'
opt.dataroot   = './datasets/'
opt.which_epoch= 22
opt.netG       = 'local'
opt.ngf        = 32
opt.label_nc   = 0
opt.no_instance= True
opt.resize_or_crop='no'

In [82]:
dataset = load_data.load_dataset(opt)
model   = load_model.load_model(opt)
model

dataset [AlignedPairDataset] was loaded
150 150


In [73]:
data    = dataset[0]

In [48]:
print(
    'data size: ',dataset.dataset_size,
    '\ndataroot: ',opt.dataroot,
    '\nphase: ',opt.phase,)

data size:  150 
dataroot:  ./datasets/ 
phase:  test


In [77]:
from skimage.io import imsave
prev_frame = torch.zeros_like(data['image'])
start_from = 0
generated = []

In [94]:
for i in tqdm(range(start_from, dataset.clip_length)):
    label = data['label'][i:i+1]
    #print(label.shape)
    inst = None if opt.no_instance else data['inst'][i:i+1]

    cur_frame = model.inference(label, inst, torch.unsqueeze(prev_frame, dim=0))
    prev_frame = cur_frame.data[0]
    
    imsave('./datasets/own_dance_test/test_sync/{:05d}.png'.format(i), util.tensor2im(prev_frame))
    generated.append(util.tensor2im(prev_frame))

  0%|                                                                                          | 0/150 [00:00<?, ?it/s]


AttributeError: 'Pose2VidModel' object has no attribute 'inference'